**训练分词器是一个统计过程，它试图确定哪些子词是给定语料库的最佳选择，而用于选择子词的确切规则取决于分词算法。它是确定性的，这意味着在相同的语料库上使用相同的算法进行训练时，您总是会得到相同的结果。**

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203021730442.png)

# 更改缓存路径

In [1]:
import os

# 更改缓存路径
os.environ["HF_HOME"] = "D:/huggingface"
os.environ["HF_DATASETS_CACHE"] = "D:/huggingface/datasets"

# 设置离线模式
# 模型离线
# os.environ['TRANSFORMERS_OFFLINE'] = '1'
# 数据离线
# os.environ['HF_DATASETS_OFFLINE'] = '1'

In [2]:
from datasets import load_dataset

# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")

Reusing dataset code_search_net (D:\huggingface\datasets\code_search_net\python\1.0.0\80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
raw_datasets["train"]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [4]:
print(raw_datasets["train"][123456]["whole_func_string"])

def last_rate_limit(self):
        """
        A `dict` of the rate limit information returned in the most recent
        response, or `None` if no requests have been made yet.  The `dict`
        consists of all headers whose names begin with ``"RateLimit"`` (case
        insensitive).

        The DigitalOcean API specifies the following rate limit headers:

        :var string RateLimit-Limit: the number of requests that can be made
            per hour
        :var string RateLimit-Remaining: the number of requests remaining until
            the limit is reached
        :var string RateLimit-Reset: the Unix timestamp for the time when the
            oldest request will expire from rate limit consideration
        """
        if self.last_response is None:
            return None
        else:
            return {k:v for k,v in iteritems(self.last_response.headers)
                        if k.lower().startswith('ratelimit')}


In [5]:
# 执行以下操作将创建一个包含 1,000 个文本的列表，但会将所有内容加载到内存中：
# training_corpus = [raw_datasets["train"][i: i + 1000]["whole_func_string"] for i in range(0, len(raw_datasets["train"]), 1000)]

# python生成器

* 在Python中，一边循环一边计算的机制，称为生成器：generator。
* **我又想要得到庞大的数据，又想让它占用空间少，那就用生成器！**
* 方法一，只要把一个列表生成式的[]改成()，就创建了一个generator。
* 方法二， 如果一个函数中包含yield关键字，那么这个函数就不再是一个普通函数，而是一个generator。调用函数就是创建了一个生成器（generator）对象。

# 可迭代类型与迭代器
* 凡是可作用于for循环的对象都是Iterable类型；
* 凡是可作用于next()函数的对象都是Iterator类型，它们表示一个惰性计算的序列；
* 集合数据类型如list、dict、str等是Iterable但不是Iterator，不过可以通过iter()函数获得一个Iterator对象。

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203021557765.png)
* 迭代器实现了两个方法，一个是__next__，一个是__iter__，__iter__用来返回迭代器本身，__next__用来取出下一个元素。
* 生成器也是迭代器的一种，迭代器只能记住自身的执行状态并等待下一次迭代，而生成器除了也会记住执行状态，还可以通过yield语句控制使多个生成器切换执行，例如手枪只能由一个枪口打出子弹，而加特林机枪可以通过旋转控制（yield切换）用多个枪口依次出子弹，威力也会更强，生成器是实现异步协程的重要基础。


In [6]:
training_corpus = (
    raw_datasets["train"][i : i + 1000]["whole_func_string"]
    for i in range(0, len(raw_datasets["train"]), 1000)
)

## 生成器只能使用一次，所以我们需要一个返回生成器的函数

In [7]:
gen = (i for i in range(10))
print(list(gen))
print(list(gen))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[]


## 为了节约内存，使用生成器加载数据

In [8]:
def get_training_corpus():
    return (
        raw_datasets["train"][i : i + 1000]["whole_func_string"]
        for i in range(0, len(raw_datasets["train"]), 1000)
    )


training_corpus = get_training_corpus()

In [9]:
# 该函数与上面的函数产生相同的生成器，在for循环中使用yield来产生生成器
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

# 训练新的分词器

## 新的分词器只有词汇表和原来的不一样，其他都和GPT-2的分词器一样

In [10]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [11]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokens = old_tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'n',
 'umbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`',
 '."',
 '""',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

**原始的GPT-2的分词器有一些特殊符号，如 Ċ 和 Ġ，分别表示空格和换行符。正如我们看到的，标记器为每个空格返回单独的标记，而实际上编程时，4个8个等空格组合在一起表示特殊的含义。GPT-2的分词器还拆分了函数名，单独标记了函数名中的_符号。**

## 我们训练一个新的分词器解决上述问题

In [12]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203021732824.png)

* AutoTokenizer.train_new_from_iterator()仅当使用的分词器是“快速”分词器时才有效。
* 🤗 Transformers 库包含两种类型的分词器：一些是纯粹用 Python 编写的，另一些（快速的）由 🤗 Tokenizers 库支持，它是用Rust编程语言编写的。
* 大多数 Transformer 模型都有一个快速标记器可用（您下面的链接查看），并且AutoTokenizer API 始终为您选择快速标记器（如果可用）。
* https://huggingface.co/docs/transformers/index#supported-frameworks

In [13]:
tokens = tokenizer.tokenize(example)
tokens

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']

在这里，我们再次看到了表示空格和换行符的特殊符号Ċ，Ġ但我们也可以看到，我们的分词器学习了一些高度特定于 Python 函数语料库的分词：例如，有一个ĊĠĠĠ表示缩进的分词，以及一个表示缩进的分词。Ġ"""表示开始文档字符串的三个引号的标记。标记器还正确地将函数名称拆分为_. 这是一个非常紧凑的表示；相比之下，在同一个例子中使用简单的英语分词器会给我们一个更长的句子。

In [14]:
print(len(tokens))
print(len(old_tokenizer.tokenize(example)))

27
36


In [15]:
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
tokenizer.tokenize(example)

['class',
 'ĠLinear',
 'Layer',
 '():',
 'ĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'init',
 '__(',
 'self',
 ',',
 'Ġinput',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'weight',
 'Ġ=',
 'Ġtorch',
 '.',
 'randn',
 '(',
 'input',
 '_',
 'size',
 ',',
 'Ġoutput',
 '_',
 'size',
 ')',
 'ĊĠĠĠĠĠĠĠ',
 'Ġself',
 '.',
 'bias',
 'Ġ=',
 'Ġtorch',
 '.',
 'zeros',
 '(',
 'output',
 '_',
 'size',
 ')',
 'ĊĊĠĠĠ',
 'Ġdef',
 'Ġ__',
 'call',
 '__(',
 'self',
 ',',
 'Ġx',
 '):',
 'ĊĠĠĠĠĠĠĠ',
 'Ġreturn',
 'Ġx',
 'Ġ@',
 'Ġself',
 '.',
 'weights',
 'Ġ+',
 'Ġself',
 '.',
 'bias',
 'ĊĠĠĠĠ']

上面的例子除了缩进对应的记号，这里我们还可以看到双缩进的记号ĊĠĠĠĠĠĠĠ：特殊的 Python 单词，如class, init, call, self, 和return每个都被标记为一个标记，我们可以看到，除了分割开_和.标记器正确分割甚至骆驼大小写的名称：LinearLayer被标记为["ĠLinear", "Layer"].

# 保存自己训练的分词器

In [16]:
tokenizer.save_pretrained("code-search-net-tokenizer")

('code-search-net-tokenizer\\tokenizer_config.json',
 'code-search-net-tokenizer\\special_tokens_map.json',
 'code-search-net-tokenizer\\vocab.json',
 'code-search-net-tokenizer\\merges.txt',
 'code-search-net-tokenizer\\added_tokens.json',
 'code-search-net-tokenizer\\tokenizer.json')

# 加载自己训练的分词器

In [17]:
tokenizer = AutoTokenizer.from_pretrained("./code-search-net-tokenizer")

# 完整代码示例

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("code_search_net", "python")

Reusing dataset code_search_net (C:\Users\ls\.cache\huggingface\datasets\code_search_net\python\1.0.0\80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]

In [5]:
from transformers import AutoTokenizer

training_corpus = get_training_corpus()
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [6]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

new_tokenizer.tokenize(example)

['def',
 'Ġadd',
 '_',
 'numbers',
 '(',
 'a',
 ',',
 'Ġb',
 '):',
 'ĊĠĠĠ',
 'Ġ"""',
 'Add',
 'Ġthe',
 'Ġtwo',
 'Ġnumbers',
 'Ġ`',
 'a',
 '`',
 'Ġand',
 'Ġ`',
 'b',
 '`."""',
 'ĊĠĠĠ',
 'Ġreturn',
 'Ġa',
 'Ġ+',
 'Ġb']